In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle
!cp /content/drive/My\ Drive/kaggle.json ~/.kaggle/

# **Project 3-** Multi-variate Time Series Forecasting



In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/GE.csv')
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-04-20,79.249802,80.183556,79.058258,79.616920,79.268036,8661237
1,2023-04-21,79.792496,79.872307,79.114128,79.417397,79.069382,5112741
2,2023-04-24,79.473267,80.422981,79.473267,79.928169,79.577919,7833505
3,2023-04-25,81.364723,81.412613,77.853149,78.563446,78.219170,11649768
4,2023-04-26,78.395851,78.571426,76.687950,76.783722,76.447243,8032357
...,...,...,...,...,...,...,...
247,2024-04-15,157.199997,158.690002,152.690002,153.699997,153.699997,6423700
248,2024-04-16,152.699997,157.020004,152.000000,156.759995,156.759995,6187200
249,2024-04-17,158.000000,158.679993,153.630005,155.669998,155.669998,6519900
250,2024-04-18,156.679993,158.000000,152.880005,152.940002,152.940002,6186600


In [ ]:
df_date =  pd.to_datetime(df['Date'])
df_train = df.drop(columns=['Date'])

In [ ]:
scaler = StandardScaler()
df_train = scaler.fit_transform(df_train)
df_train

array([[-1.01498383, -1.00024983, -0.99735778, -0.99853348, -1.00043708,
         0.70045746],
       [-0.98750629, -1.01561708, -0.99446976, -1.00856896, -1.01040695,
        -0.51476829],
       [-1.00366941, -0.98842873, -0.97590524, -0.98287847, -0.98488494,
         0.41699053],
       ...,
       [ 2.97227651,  2.87534347,  2.85738569,  2.82673547,  2.83395677,
        -0.03286952],
       [ 2.90544224,  2.84177028,  2.81861688,  2.68942389,  2.69694616,
        -0.14701217],
       [ 2.74190189,  2.62107356,  2.50329695,  2.44397257,  2.45203285,
         1.00955117]])

In [ ]:
X_train = []
y_train = []
future = 1
past = 14
for i in range(past,len(df_train)+1 - future):
    X_train.append(df_train[i-past: i,0:df_train.shape[1]])
    y_train.append(df_train[i+future-1:i+future,0])

X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
y_train.shape

(238, 1)

In [ ]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1]))

model.compile(optimizer='adam', loss='mse', metrics=['acc'])
model.summary()

# fit the model
history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.1, verbose=1)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 14, 64)            18176     
                                                                 
 lstm_5 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 30625 (119.63 KB)
Trainable params: 30625 (119.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
14/14 [==============================] - 4s 54ms/step - loss: 0.3722 - acc: 0.0000e+00 - val_loss: 0.2666 - val_acc: 0.0000e+00
Epoch 2/1

In [ ]:
predicted_values = []

# Predict each day starting from day 15
for i in range(0, len(X_train)):
    X_pred = X_train[i].reshape((1, X_train.shape[1], X_train.shape[2]))
    predicted_value = model.predict(X_pred)
    predicted_values.append(predicted_value)

# Convert list of predictions to numpy array
predicted_values = np.array(predicted_values)

In [ ]:
print('Predicted: ',(predicted_values)[5:10])
print('y_train: ',(y_train)[5:10])

Predicted:  [[[-0.89563686]]

 [[-0.8889375 ]]

 [[-0.8746963 ]]

 [[-0.8608282 ]]

 [[-0.8434057 ]]]
y_train:  [[-0.91113416]
 [-0.86991779]
 [-0.80405196]
 [-0.82102357]
 [-0.83072138]]


In [ ]:
comparison = np.abs(predicted_values - y_train)  # y_train starts from index 14

# Calculate mean absolute error
mae = np.mean(comparison)

print("Mean Absolute Error:", mae)

Mean Absolute Error: 1.0476522807089723
